In [1]:
#imports
import requests 
from bs4 import BeautifulSoup 
import pandas as pd
import time

In [2]:
#variables
scrappingUrl = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
urlContent = requests.get(scrappingUrl)

In [4]:
soup = BeautifulSoup(urlContent.text)
theadraw=soup.select('table', class_='stats_table sortable min_width force_mobilize now_sortable')
theadraw
soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/fb/deploy/www/base" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://cdn.ssref.net/req/202211181" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://cmp.quantcast.com'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, 
		    '/choice.js?tag_version=V2');
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function makeStub() {
	    var TCF_LOCATOR_NAME = '_

In [15]:
theadraw=soup.select('table', class_='stats_table sortable min_width force_mobilize now_sortable')[0]
theadraw = theadraw.find_all('td',class_='left')
teams =[]
for i in range(0,len(theadraw),3):
    teams.append(theadraw[i].find('a'))

teams = [f"https://fbref.com{ln.get('href')}" for ln in teams ]
print(teams) 

IndexError: list index out of range

In [7]:
Allteams_dataframe = pd.DataFrame()

In [5]:
for i in range(0,len(teams)):
    print(i)
    urlContent = requests.get(teams[i])
    matchStats = pd.read_html(urlContent.text, match="Scores & Fixtures")[0]
    soup = BeautifulSoup(urlContent.text)
    links = soup.find_all('a')
    shooting_link=[l.get('href') for l in links]
    shooting_link=[l for l in shooting_link if l and 'all_comps/shooting/' in l]
    shooting_link = [f"https://fbref.com{l}" for l in shooting_link ]
    shooting_url = shooting_link[0]
    shooting_dataFrame = pd.read_html(shooting_url,match = 'Shooting')[0]
    shooting_dataFrame.columns = shooting_dataFrame.columns.droplevel()
    shooting_dataFrame
    combine_frame = matchStats.merge(shooting_dataFrame[['Date','Sh','SoT','Dist','FK','PK','PKatt','G/Sh','G/SoT','xG']],on='Date')
    Allteams_dataframe=pd.concat([Allteams_dataframe,combine_frame])
    time.sleep(10)
Allteams_dataframe

NameError: name 'teams' is not defined

In [ ]:
Allteams_dataframe.to_csv('matches.cvs')